In [41]:
import csv
import re
import pandas as pd

# Reading in data
in_file = open("data/district_data_23_Billion.csv", 'r')
out_file = open("data/gap_in_spending_per_student.csv", 'w')
writer = csv.writer(out_file)
data = list(csv.reader(in_file))

# Initializing lists/dictionaries
state_revenue_list = []
final = {}
lower = {}
higher = {}

# Getting state and student revenue columns
for i in data[2:]:
    state = i[0]
    revenue = i[13]
    revenue = re.sub("[^0-9]", "", revenue)
    state_revenue_list.append((state,revenue))

# Initializing lower and higher lists
for i in state_revenue_list:
    if i[0] not in lower:
        lower[i[0]] = i[1]
    if i[0] not in higher:
        higher[i[0]] = i[1]
    
# Finding highest and lowest student revenue in each state
for i in state_revenue_list:
    if int(i[1]) > int(higher[i[0]]):
        higher[i[0]] = int(i[1])
    if int(i[1]) < int(lower[i[0]]):
        lower[i[0]] = int(i[1])

# Finding difference
for i in lower:
    final[i] = int(higher.get(i)) - int(lower.get(i))

# Writing to new file
final.pop("District of Columbia")
writer.writerow(["state", "gap_in_spending_per_student"])
for key, value in final.items():
    writer.writerow([key, value])

# Adding state codes to CSV
df = pd.read_csv("data/gap_in_spending_per_student.csv")
state_code = pd.Series(["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"])
df.insert(0,"code",state_code)
df.to_csv("data/gap_in_spending_per_student.csv", index=False)

 $	8,448 
8448
 $	9,520 
9520
 $	9,852 
9852
 $	9,554 
9554
 $	8,746 
8746
 $	9,865 
9865
 $	8,725 
8725
 $	16,636 
16636


KeyError: 'District of Columbia'

In [46]:
# Reading in data
in_file = open("data/district_data_23_Billion.csv", 'r')
out_file = open("data/median_property_value.csv", 'w')
writer = csv.writer(out_file)
data = list(csv.reader(in_file))

# Initializing lists/dictionaries
state_revenue_list = []
final = {}
lower = {}
higher = {}

# Getting state and student revenue columns
for i in data[2:]:
    state = i[0]
    value = i[20]
    value = re.sub("[^0-9]", "", value)
    state_revenue_list.append((state,value))

# Initializing lower and higher lists
for i in state_revenue_list:
    if i[0] not in lower:
        lower[i[0]] = i[1]
    if i[0] not in higher:
        higher[i[0]] = i[1]
    
# Finding highest and lowest student revenue in each state
for i in state_revenue_list:
    
    if int(i[1]) > int(higher[i[0]]):
        higher[i[0]] = int(i[1])
    if int(i[1]) < int(lower[i[0]]):
        lower[i[0]] = int(i[1])

# Finding difference
for i in lower:
    final[i] = int(higher.get(i)) - int(lower.get(i))

# Writing to new file
final.pop("District of Columbia")
writer.writerow(["state", "median_property_value"])
for key, value in final.items():
    writer.writerow([key, value])

# Adding state codes to CSV
df = pd.read_csv("data/median_property_value.csv")
state_code = pd.Series(["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"])
df.insert(0,"code",state_code)
df.to_csv("data/median_property_value.csv", index=False)

106200
107500
97900
98500
149300
139400
72200
226600
141000
173400
73600
85300
97200
87100
122800
107600
97400
68000
83000
103500
83100
112500
78200
99100
64600
84200
90000
109600
106100
94700
72300
76500
94700
77600
143400
112700
103200
111600
73900
127100
96800
115100
141000
103300
154100
166500
92500
118400
126900
95100
75300
126000
115600
85200
72700
86300
77700
68100
167500
89400
72700
131600
101800
299100
266700
96200
168900
95200
149400
136000
133200
71100
81200
123300
97600
122500
141600
136400
83500
68900
77000
233600
160400
78500
79400
110400
75600
122600
87700
118100
120400
557300
135000
153500
153400
87900
126600
98400
148600
73700
124100
89100
78200
79400
244900
88300
78500
119400
96400
135600
152000
128600
90400
88900
188200
139600
73400
117000
83500
87100
104000
105800
72900
107100
150900
233500
170400
155900
102300
346600
76200
81100
81600
116600
79900
126700
52500
124500
298000
90300
199000
251000
98200
155100
299500
210900
237200
228600
255500
224000
117400
198400
167

ValueError: invalid literal for int() with base 10: ''